### El objetivo de este test es ver si realmente mejora RSA al aplicarle un tunner

In [79]:
import pandas as pd
import numpy as np
from mealpy.tuner import Tuner
import benchmark_functions as bf
from RSA import RSA
from tqdm import tqdm
import pickle

In [70]:
# PARAMETROS UTILIZADOS

dims = 5
funciones = [
  bf.Schwefel(n_dimensions=dims),
  bf.Ackley(n_dimensions=dims),
  bf.Michalewicz(n_dimensions=dims),
  bf.Rastrigin(n_dimensions=dims),
  bf.Rosenbrock(n_dimensions=dims),
  bf.DeJong3(n_dimensions=dims),
  bf.StyblinskiTang(n_dimensions=dims),
  bf.EggHolder(n_dimensions=dims),
  bf.Rana(n_dimensions=dims),
]

In [71]:
# PRIMERO RESOLVER PROBLEMA SIN TUNNER
def evaluate_functions(epochs=1000, pop_size=100, best_params=None):
  resultados = []

  for func in tqdm(funciones):
      resultados_fila = [func.name()]

      LB,UB = func.suggested_bounds()
      problem = {
          "fit_func": lambda x: func(list(x)),
          "lb": LB,
          "ub": UB,
          "minmax": "min",
          "log_to": None,
          "save_population": False,
          "verbose": False
      }

      if type(best_params) == pd.core.frame.DataFrame:
        alpha = best_params[best_params['Funcion']==func.name()]['alpha'].values[0]
        beta = best_params[best_params['Funcion']==func.name()]['beta'].values[0]
      else:
         alpha = 0.1
         beta = 0.1

      r= []
      for i in range(10):
        model = RSA(epochs,pop_size)
        best_position, best_fitness = model.solve(problem)
        r.append( best_fitness )
      resultados_fila.append( np.min(r) )
      resultados_fila.append( np.mean(r) )
      resultados.append(resultados_fila)

  col_names = ['Funcion','min','mean']
  df = pd.DataFrame(resultados,columns=col_names)
  return df

In [72]:
# TUNNERAR

best_params = []

paras_bbo_grid = {
    "epoch": [100],
    "pop_size": [70],
    "alpha": [0.005, 0.01, 0.05, 0.1, 0.5],
    "beta": [0.001,0.005, 0.01, 0.05, 0.1, 0.5]
}

for func in tqdm(funciones):
    best_params_fila = [func.name()]

    LB,UB = func.suggested_bounds()
    problem = {
            "fit_func": func,
            "lb": LB,
            "ub": UB,
            "minmax": "min",
            "log_to": None,
            "save_population": False,
            "verbose": False
    }

    model = RSA()
    tuner = Tuner(model, paras_bbo_grid)
    tuner.execute(problem=problem, n_trials=10, mode="parallel", n_workers=4,verbose=False)
    best_params_fila.append( tuner.best_params['alpha'] )
    best_params_fila.append( tuner.best_params['beta'] )

    best_params.append(best_params_fila)

col_names = ['Funcion','alpha','beta']
best_params = pd.DataFrame(data=best_params,columns=col_names)
best_params

100%|██████████| 9/9 [18:14<00:00, 121.66s/it]


,Funcion,alpha,beta
0,Schwefel,0.050,0.005
1,Ackley,0.005,0.001
2,Michalewicz,0.100,0.050
3,Rastrigin,0.005,0.001
4,Rosenbrock,0.100,0.001
5,De Jong 3,0.005,0.500
6,Styblinski and Tang,0.050,0.010
7,Egg Holder,0.010,0.100
8,Rana,0.010,0.100


In [82]:
# SAVE TUNING
with open('tuning.pickle','wb') as f:
    pickle.dump(best_params,f)

In [73]:
sin_tune = evaluate_functions(1000,100)
sin_tune

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [10:52<00:00, 72.52s/it]


,Funcion,min,mean
0,Schwefel,2.772924e+02,3.895866e+02
1,Ackley,4.440892e-16,4.440892e-16
2,Michalewicz,-4.629753e+00,-4.238464e+00
3,Rastrigin,0.000000e+00,0.000000e+00
4,Rosenbrock,8.718482e-01,2.848223e+00
5,De Jong 3,-2.000000e+01,-1.960000e+01
6,Styblinski and Tang,-1.956799e+02,-1.758799e+02
7,Egg Holder,-2.962114e+03,-2.602204e+03
8,Rana,-1.798661e+03,-1.639425e+03


In [75]:
con_tune = evaluate_functions(1000,100,best_params)
con_tune

,Funcion,min,mean
0,Schwefel,2.397158e+02,4.127787e+02
1,Ackley,4.440892e-16,4.440892e-16
2,Michalewicz,-4.590339e+00,-4.189966e+00
3,Rastrigin,0.000000e+00,0.000000e+00
4,Rosenbrock,3.325588e-02,2.601721e+00
5,De Jong 3,-2.000000e+01,-1.940000e+01
6,Styblinski and Tang,-1.958000e+02,-1.794197e+02
7,Egg Holder,-2.789265e+03,-2.572480e+03
8,Rana,-1.791450e+03,-1.589873e+03


Se observa que con tuner se obtienen mejores resultados en un 6 de las 9 funciones :D para el caso min y cercanas a la esperanza (4.5) para el caso de mean

In [78]:
con_tune[['min','mean']] <= sin_tune[['min','mean']]

,min,mean
0,True,False
1,True,True
2,False,False
3,True,True
4,True,True
5,True,False
6,True,True
7,False,False
8,False,False
